In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from mne.datasets import eegbci
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, Activation, Dropout, Conv2D,
    DepthwiseConv2D, SeparableConv2D, BatchNormalization,
    AveragePooling2D, Input, Flatten
)
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import load_model
import pickle

# =============================================================================
# 1. MEMUAT DATASET EDF DARI PHYSIONET
#    Ini adalah dataset yang berbeda dari BCI Competition.
# =============================================================================
print("Mengunduh dan memuat dataset PhysioNet EDF...")

subjects = range(1, 21)
all_X = []
all_y = []
# Run untuk tugas motor imagery (membayangkan kepalan tangan kiri/kanan)
runs = [6, 10, 14]

for subject in subjects:
  print(f"Memuat data subjek {subject}...")
  raw_files = eegbci.load_data(subject, runs, update_path=True, verbose=False)

  # Membaca file .edf. Kali ini, tidak akan ada konflik format.
  raw = mne.io.concatenate_raws([mne.io.read_raw_edf(f, preload=True, verbose='WARNING') for f in raw_files])

  # =============================================================================
  # 2. PREPROCESSING
  # =============================================================================
  print("Melakukan preprocessing data...")

  # Memilih 64 channel EEG yang tersedia di dataset ini
  raw.pick_types(meg=False, eeg=True, stim=False, eog=False, exclude='bads')
  # Removing the problematic set_montage line
  # raw.set_montage('standard_1005', on_missing='warn') # Removed

  # Menerapkan band-pass filter
  raw.filter(l_freq=4.0, h_freq=38.0, fir_design='firwin', skip_by_annotation='edge')

  # Mendefinisikan 2 kelas motor imagery: Tangan Kiri (T1) dan Tangan Kanan (T2)
  event_id = dict(T1=1, T2=2)
  events, _ = mne.events_from_annotations(raw, event_id=event_id)

  # Membuat epoch di sekitar setiap cue
  tmin, tmax = -1., 4.
  epochs = mne.Epochs(
      raw, events, event_id=event_id, tmin=tmin, tmax=tmax,
      proj=True, baseline=None, preload=True, verbose=False
  )

  # =============================================================================
  # 3. EKSTRAKSI DATA & FORMATTING
  # =============================================================================
  X = epochs.get_data()
  # Mengubah label dari [1,2] menjadi [0,1] untuk model kita
  y = epochs.events[:, -1] - 1
  all_X.append(X)
  all_y.append(y)

X = np.concatenate(all_X, axis=0)
y = np.concatenate(all_y, axis=0)


print("\n--- ✅ Proses Selesai! Data Berhasil Dimuat! ---")
print(f"Bentuk data (X): {X.shape}")
print(f"Bentuk label (y): {y.shape}")
print(f"Label unik: {np.unique(y)}")
print(f"Jumlah data per kelas: {np.bincount(y)}")

# Fungsi untuk memplot riwayat pelatihan
def plot_history(history):
    # Plot Akurasi
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()


def EEGNet(nb_classes, Chans=64, Samples=128, dropoutRate=0.6,
                      kernLength=64, F1=8, D=2, F2=16):
    """
    Kerangka arsitektur EEGNet yang harus Anda lengkapi.

    Args:
      nb_classes: Jumlah kelas keluaran (misal: 2 untuk tangan kiri/kanan).
      Chans: Jumlah channel EEG.
      Samples: Jumlah titik waktu (sampel) dalam satu epoch.
      ... (parameter lainnya untuk mengontrol arsitektur)
    """

    # Input Layer
    input1 = Input(shape=(Chans, Samples, 1))

    ##################################################################
    # BLOCK 1: Temporal & Spatial Filtering (CONTOH LENGKAP)
    ##################################################################
    # Blok ini sudah saya lengkapi sebagai referensi Anda.
    # Perhatikan polanya: Conv -> BatchNorm -> Activation -> Pool -> Dropout

    block1 = Conv2D(F1, (1, kernLength), padding='same', use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False,
                               depth_multiplier=D,
                               depthwise_constraint=max_norm(1.))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropoutRate)(block1)

    ##################################################################
    # BLOCK 2: Separable Convolution
    ##################################################################
    # TUGAS ANDA #1: Lengkapi lapisan SeparableConv2D di bawah ini.
    # - Berapa jumlah filter yang harus digunakan? (Lihat parameter fungsi 'F2')
    # - Ukuran kernelnya adalah (1, 16).
    # - Gunakan padding='same' dan use_bias=False.

    block2 = SeparableConv2D(F2 , (1, 16) , padding='same', use_bias=False)(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropoutRate)(block2)

    ##################################################################
    # CLASSIFICATION BLOCK
    ##################################################
    flatten = Flatten(name='flatten')(block2)

    # TUGAS ANDA #2: Lengkapi lapisan Dense terakhir untuk klasifikasi.
    # - Berapa jumlah neuron yang dibutuhkan untuk output? (Terkait dengan 'nb_classes')
    # - Fungsi aktivasi apa yang cocok untuk klasifikasi multi-kelas? (Hint: 'softmax')

    dense = Dense( nb_classes , name='dense', kernel_constraint=max_norm(0.25), activation='softmax' )(flatten)

    return Model(inputs=input1, outputs=dense)


# =============================================================================
# 4. PERSIAPAN DATA UNTUK TENSORFLOW
# =============================================================================
y_cat = to_categorical(y, num_classes=2)
X_4d = np.expand_dims(X, axis=-1)
X_train, X_test, y_train, y_test = train_test_split(X_4d, y_cat, test_size=0.2, random_state=42)

# =============================================================================
# 2. MEMBUAT DAN MELATIH MODEL
# =============================================================================

# Ambil jumlah channel dan sampel dari data training Anda
chans = X_train.shape[1]
samples = X_train.shape[2]
num_classes = y_train.shape[1]

# Inisialisasi model EEGNet
model = EEGNet(nb_classes=num_classes, Chans=chans, Samples=samples)

# TUGAS #4: Kompilasi model.
# Gunakan optimizer 'adam' dan loss 'categorical_crossentropy'.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# TUGAS #5: Latih model dengan model.fit().
# Latih selama 50 epoch dengan batch_size 8.
# Gunakan data test sebagai validation_data.
history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=50,
    validation_data=(X_test, y_test) # Gunakan data test sebagai validasi
)

print("\n--- ✅ Pelatihan Selesai! ---")
print(f"\n Menyimpan model ke direktori ")
model.save('eegnet_model_final.h5')
print("\nModel berhasil disimpan ke 'eegnet_model_final.h5'")

with open('training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)
print("Riwayat pelatihan berhasil disimpan ke 'training_history.pkl'")


# =============================================================================
# 3. ANALISIS HASIL
# =============================================================================
# Gunakan kembali fungsi plot_history yang sudah kita buat untuk melihat hasilnya.
plot_history(history)